![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&urlpath=Mathematics/CurveFitting/fitting-co2-data.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"></a>

# Fitting Functions to Real Data

In this notebook, we will try to fit functions to real-world data. You may want to first check out the [Curve Fitting](./curve-fitting.ipynb) notebook.

We'll start with some data from [Baxters Harbour](https://www.google.com/maps/place/Baxters+Harbour) in the [Bay of Fundy](https://www.waterlevels.gc.ca/eng/find/zone/30), an area famous for its unusually high tides.

In [ ]:
import requests
import pandas as pd
import plotly.express as px

# Get tide data from Fisheries and Oceans Canada hourly for the next 7 days
url = 'https://www.waterlevels.gc.ca/eng/station?sid=305'
response = requests.get(url)

# construct a dataframe from the 7th table on the webpage
tides_data = pd.read_html(response.text)[7]['Hour'].transpose()

# since there are multiple columns of data, construct a list from tides_data
tide_list = []
for key in tides_data.keys():
    tide_list.extend(tides_data[key].tolist())

# turn the list into a dataframe, label the height column and create a time column
tides = pd.DataFrame(tide_list)
tides.columns = ['Height (m)']
tides['Time (h)'] = range(0,len(tide_list))

# display the dataframe
tides

We can create an interactive visualization of the data using the [Plotly Express](https://plotly.com/python/plotly-express) library.

In [ ]:
fig = px.line(tides, x='Time (h)', y='Height (m)', title='Baxters Harbour Water Levels')
fig.show()

We can see periodic variation in the water level due to the tides. Notice that we are using `px.line` instead of `px.scatter` in order to connect the data points and allow us to determine the type of function that would fit the data.

It looks like a sine wave, let's try to find a good curve fit using the function $h=a\sin(bt+c)+d$.

* $a$ = amplitude, the maximum distance from the center of the wave
* $b$ = frequency, how often the wave peaks
* $c$ = phase shift, the horizontal offset of the wave (between 0 and 2$\pi$)
* $d$ = vertical shift

We can calculate the amplitude as half of the distance between maximum and minimum height.

In [ ]:
(tides['Height (m)'].max() - tides['Height (m)'].min()) / 2

And the vertical shift should be approximately equal to the mean height value.

In [ ]:
tides['Height (m)'].mean()

Adjust the values of $a$, $b$, $c$, and $d$ to see if you can get a reasonable fit to the data.

In [ ]:
a = 5.8
b = 0.5
c = 1
d = 6.5

x_values = tides['Time (h)']
y_values = tides['Height (m)']

from scipy.optimize import curve_fit
import numpy as np
tides['Manual Curve Fit'] = a * np.sin(b * x_values + c) + d

def tide_fit(x, a, b, c, d):
    return a * np.sin(b * x + c) + d
values, fit_quality = curve_fit(tide_fit, x_values, y_values)
print('Values:', values)
print('Uncertainty:', fit_quality)
e, f, g, h = values
tides['SciPy Curve Fit'] = e * np.sin(f * x_values + g) + h

fig1 = px.scatter(tides, x='Time (h)', y='Height (m)', title='Tide Levels With Curve Fit')
fig1.add_scatter(x=tides['Time (h)'], y=tides['Manual Curve Fit'], name='Manual Curve Fit')
fig1.add_scatter(x=tides['Time (h)'], y=tides['SciPy Curve Fit'], name='SciPy Curve Fit')
fig1.show()

Unfortunately in this case there weren't enough data points for SciPy's `curve_fit` to do a good job, but it did recognize the period variation in the data.

Let's look at a data set with more values.

## Atmospheric Carbon Dioxide

Next we will download measurements of carbon dioxide in the atmosphere from 1958 to 2019, then fit a function to this data in order to perhaps make predictions about future atmospheric $CO_2$ concentrations.

In [ ]:
# A file containing the measurements
#url = "ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_mm_mlo.txt"
url = "co2_mm_mlo.txt" # the file stored locally
# Download the file, ignore the headers, and specify the data is separated by spaces
co2 = pd.read_csv(url, skiprows=72, sep='\s+').reset_index(drop=True)
# Rename the columns to something more convenient 
co2.columns = ["Year", "Month", "Date", "Carbon Dioxide (ppm)", "Interpolated", "Trend", "Number of Days"]
# Missing data is filled in with -99.99, so we get rid of it 
co2 = co2[~co2['Carbon Dioxide (ppm)'].isin([-99.99])]
# View the data
co2

The only data we need to use is the `Date` column (in [decimal time](https://en.wikipedia.org/wiki/Decimal_time)) and the `Carbon Dioxide (ppm)` column. Let's create an interactive plot of those two columns.

In [ ]:
fig2 = px.scatter(co2, x='Date', y='Carbon Dioxide (ppm)', title='Atmospheric Carbon Dioxide Over Time')
fig2.show()

## Try It Yourself

You probably notice that there's an increasing trend and some periodic behavior. Your task is to fit a function to this data.

Feel free to make _any_ transformations you see fit to the data and fit _any_ function you wish to the data as well. Feel free to try multiple transformations. For example, if you wanted to take the square root of the entire column, you could use `co2['Curve Fit'] = np.sqrt(x_data)` to create a new column with the square roots of the `x_data` column.

Below we set up some starter code for you to get working on this task. Remember that you can refer to [this curve fit notebook](./curve-fitting.ipynb) for examples. Good Luck!

In [ ]:
x_data = co2['Date']
y_data = co2['Carbon Dioxide (ppm)']

# Create your function to fit here
def co2_fit(x, a, b):
    return a * x + b

values, fit_quality = curve_fit(co2_fit, x_data, y_data)
print('Values:', values)
print('Uncertainty:', fit_quality)

# Change this to your fit function as well
a, b = values
co2['Curve Fit'] = a * x_data + b

fig3 = px.scatter(co2, x='Date', y='Carbon Dioxide (ppm)', title='Atmospheric Carbon Dioxide Over Time')
fig3.add_scatter(x=co2['Date'], y=co2['Curve Fit'], name='Curve Fit')
fig3.show()

# Conclusion

Hopefully you were able to find a decent curve fit for the data. For more information about `curve_fit` check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html).

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)